**Used Car Price Prediction 
Step1 : Exploratory Data Analysis**
In this project, we have a look into the data related to used cars and understand the relationships among various variables and figure out the features which have a maximum impact on price. We use Correlation analysis, visualization techniques, log of features, as well as polynomial features, outlier analysis, normal distribution analysis etc. The goal is to create a predictive model which can give an estimation of the price to a customer who wants to buy/sell a used car. The data is taken from https://www.kaggle.com/CooperUnion/cardataset,

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from pandas.tools.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from collections import OrderedDict

import os
print(os.listdir("../input"))
carSales = pd.read_csv('../input/data.csv',sep=",")
carSales.head()
#Lets import the dataset and have a look at the first few records.
carSales.info()
carSales['Vehicle Style'].value_counts()
carSales['Vehicle Size'].value_counts()
#As we can see above, there are a total of 16 features. The popularity of car is a number obtained from twitter assigned to a particular 
#Make and Model. Since, the users will not be aware of the popularity of a particular make, we should choose to ignore this feature.
#dropping Popularity
carSales=carSales.drop('Popularity',axis=1)

#We create derived feature,the Age of the car, derived from the Year of Mfr., which is certainly 
#a very important characteristic that a buyer looks into while looking for a used car.
#This dataset is from 2017. So, using that as a reference to calculate the age of the car .
print(carSales['Year'].max())
carSales['Age']=2017-carSales['Year']

carSales.describe()

ax=plt.hist(carSales['MSRP'],bins=50)
plt.xlim(0,150000)
plt.show()
%pylab inline
Make=carSales.groupby(['Make'])['MSRP'].median()
Make.plot(kind='bar',stacked=True)
pylab.ylabel('Median MSRP')
pylab.title('Chart: Median MSRP by Make')
show()



#From the above plot, we see that the Make of a car has a significant impact on price. This is obvious as different manufacturers produce cars in different price ranges. It might be difficult for a single model to fit well to all the data. So, it might be wise to divide the data w.r.t price and then we can have individual models for a single price range.
# Any results you write to the current directory are saved as output.

Clearly, this is a skewed distribution. There are cars of several makes and models in this dataset. Lets have a look at the price with respect to the Make of the car. We are choosing the Median as the Central Tendency measure instead of the Mean since the Median is more robust in nature compared to the Mean. For some models such as Mercedes Benz, there are models such as SLR McLaren which have an approximate price range of $480000. This may drive the mean towards the right of the distribution and result in us placing the Mercedes Benz in a wrong category. Hence, median is a better choice here.

From the above plot, we see that the Make of a car has a significant impact on price. This is obvious as different manufacturers produce cars in different price ranges. It might be difficult for a single model to fit well to all the data. So, it might be wise to divide the data w.r.t price and then we can have individual models for a single price range.

In [ ]:
carSales=carSales.join(carSales.groupby('Make')['MSRP'].median(), on='Make', rsuffix='_Median')
make = carSales.groupby('Make')['MSRP'].median().reset_index()
pd.options.display.float_format = '{:.4f}'.format
make.sort_values('MSRP', ascending=False)

def map_MSRP_to_group(x):
    if x<30000:
        return 'ordinary'
    elif x<60000 :
        return 'deluxe'
    elif x<90000:
        return 'super-deluxe'
    elif x<350000:
        return 'luxury'
    else:
        return 'super-luxury'
#function to convert a series    
def convert_MSRP_series_to_MSRP_group(MSRP):
    return MSRP.apply(map_MSRP_to_group)

MSRP_group=convert_MSRP_series_to_MSRP_group(carSales['MSRP_Median'])
carSales['MSRP_group'] = MSRP_group
carSales.head()

Lets have a look at the Makes present in various price groups



In [ ]:
carSales[carSales['MSRP_group']=='ordinary']['Make'].unique()

In [ ]:
carSales[carSales['MSRP_group']=='deluxe']['Make'].unique()

In [ ]:
carSales[carSales['MSRP_group']=='super-deluxe']['Make'].unique()

In [ ]:
carSales[carSales['MSRP_group']=='luxury']['Make'].unique()

In [ ]:
carSales[carSales['MSRP_group']=='super-luxury']['Make'].unique()

Lets save the data as pickle files and they can be used easily for further processing

In [ ]:
import pickle
ordinary='./ord.pkl'
deluxe='./del.pkl'
supdel='./supdel.pkl'
luxury='./luxury.pkl'
suplux='./suplux.pkl'

with open(ordinary, "wb") as f:
    w = pickle.dump(carSales[carSales['MSRP_group']=='ordinary'],f)
with open(deluxe, "wb") as f:
    w = pickle.dump(carSales[carSales['MSRP_group']=='deluxe'],f)
with open(supdel, "wb") as f:
    w = pickle.dump(carSales[carSales['MSRP_group']=='super-deluxe'],f)
with open(luxury, "wb") as f:
    w = pickle.dump(carSales[carSales['MSRP_group']=='luxury'],f)
with open(suplux, "wb") as f:
    w = pickle.dump(carSales[carSales['MSRP_group']=='super-luxury'],f)

Here, we shall provide the analysis for just one of the files. The remaining files will be processed in the same manner. Lets have a look at the ordinary category

In [ ]:
ordinary=pd.read_pickle('./ord.pkl')
ordinary.info()

The index is still set to that of the original dataframe. Lets reset that.

In [ ]:
ordinary = ordinary.reset_index(drop=True)
ordinary=ordinary.drop(['MSRP_group','MSRP_Median'],axis=1)

We know that there are a few assumtions for Linear Regression:
No or little multicollinearity
Linear relationship
Multivariate normality
No auto-correlation
Homoscedasticity 
Let us check if the above assumptions hold true for our data. First we will try to check for multicollinearity i.e whether there is any correlation between the independent variables. We will create a heatmap for correlation among the various variables.

In [ ]:
corr=ordinary.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()


In [ ]:
#Also showcasing the table below to lookup exact values for reference.
corr

As we can see above, there is a very strong negative correlation between Year and Age, which makes sense as the Age of the car is a derived feature from Year.

In [ ]:
from numpy import cov
cov(carSales['highway MPG'],carSales['city mpg'])

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(carSales['highway MPG'],carSales['city mpg'])
print('Pearsons correlation betweeen highway MPG and city MPG: %.3f' % corr)

from scipy.stats import pearsonr
corr, _ = pearsonr(carSales['highway MPG'],carSales['city mpg'])
print('Pearsons correlation betweeen highway MPG and city MPG: %.3f' % corr)

In [ ]:
#dropping the features as discussed above
ordinary=ordinary.drop(['Year','highway MPG'],axis=1)

We draw Scatter plot between Price and Engine HP. and find an interesting result.

In [ ]:
plt.scatter(ordinary['MSRP'], ordinary['Engine HP'])
plt.show()

In [ ]:
#do some data cleansing to make sure the correlation analysis runs without any error
m=ordinary["MSRP"].isnull().any()
print(m[m])
m=ordinary["Engine HP"].isnull().any()
print(m[m])
ordinary["Engine HP"].fillna(ordinary["Engine HP"].mean())
ordinary['Engine HP'] = ordinary['Engine HP'].apply(lambda x: x if not pd.isnull(x) else ordinary["Engine HP"].mean())
m=ordinary["Engine HP"].isnull().any()
print(m[m])

The relationship does not look linear. We decide to try Pearson correlation. The Pearson correlation coefficient (named for Karl Pearson) can be used to summarize the strength of the linear relationship between two data samples. We try powers of Engine HP, ex: square, cube as well as log of Engine HP

In [ ]:
from scipy.stats import spearmanr

corr_p, _ = pearsonr((ordinary['MSRP']), ordinary['Engine HP'])
print('Pearson correlation between Price and Engine HP : %.3f' % corr_p)

corr_p, _ = pearsonr((ordinary['MSRP']), np.square(ordinary['Engine HP']))
print('Pearson correlation between Price and square of Engine HP : %.3f' % corr_p)

corr_p, _ = pearsonr((ordinary['MSRP']), np.power(ordinary['Engine HP'],3))
print('Pearson correlation between Price and cube of Engine HP : %.3f' % corr_p)

corr_p, _ = pearsonr((ordinary['MSRP']), np.log(1+ordinary['Engine HP']))
print('Pearson correlation between Price and log of Engine HP : %.3f' % corr_p)

We observe that the Engine HP, as is, has the highest correlation with Price
In case of nonlinear relationship or if the two variables being considered have a non-Gaussian distribution. In this case, the Spearman’s correlation coefficient (named for Charles Spearman) can be used to summarize the strength between the two data samples

In [ ]:
corr, _ = spearmanr(ordinary['MSRP'], ordinary['Engine HP'])
print('Spearman correlation between Price and Engine HP: %.3f' % corr)
corr, _ = spearmanr(ordinary['MSRP'], np.log(1+ordinary['Engine HP']))
print('Spearman correlation between Price and log of Engine HP : %.3f' % corr)

We can also create a pairplot to better understand the relationship among features. We are selecting Make as the legend
to see if the relationships vary with Make.

In [ ]:
import seaborn as sns
sns.set(style="ticks")
sns.pairplot(ordinary,hue='Make')

We can draw some interesting insights from the charts above. For example, Number of doors and Engine Cylinders are better suited as categorical features rather than numerical. Also, the relationship of variables such as City MPG and Age with MSRP are not exactly linear. We can try some feature transformations to fix that.
We will explore these in detail going forward

**Data Cleansing**
Next, we try to fix the blank values. As, we can see above there are blank values for Engine Fuel Type,Market Category and Engine Cyliders. Now lets check the Engine Fuel Type first

In [ ]:
ordinary[ordinary['Engine Fuel Type'].isnull()]

We lookup the values from the other records in the dataset or we check the internet. For eg, we check for Suzuki Verona as below.

In [ ]:
ordinary[(ordinary['Model']=='Verona')&(ordinary['Make']=='Suzuki')]

In [ ]:
#For Suzuki Verona,  Fuel Type is regular unleaded
ordinary.loc[(ordinary['Engine Fuel Type'].isnull())&(ordinary['Model']=='Verona')&(ordinary['Make']=='Suzuki'),'Engine Fuel Type']='regular unleaded'

Now, for engine cylinders, the makes and models having NaN's genuinely have no cylinders, only rotors. We replace the NaN's with zeros.

In [ ]:
ordinary.loc[(ordinary['Make']=='Mazda')&(ordinary['Model']=='RX-8'),'Engine Cylinders']=0
ordinary.loc[(ordinary['Make']=='Mazda')&(ordinary['Model']=='RX-7'),'Engine Cylinders']=0
ordinary.loc[(ordinary['Make']=='Mitsubishi')&(ordinary['Model']=='i-MiEV'),'Engine Cylinders']=0
ordinary.loc[(ordinary['Make']=='Chevrolet')&(ordinary['Model']=='Bolt EV'),'Engine Cylinders']=0
ordinary.loc[(ordinary['Make']=='Volkswagen')&(ordinary['Model']=='e-Golf'),'Engine Cylinders']=0

Market Category contains some generic categorical value for the car. The buyer/seller will not be aware of such values in our dataset. Hence, it should not be considered while creating the regression model

In [ ]:
ordinary=ordinary.drop('Market Category',axis=1)

**Normality and Outliers**
Now, lets check for normality. We can check this by creating histogram of the features.

In [ ]:
plt.hist(ordinary['Age'])
plt.title('Age Histogram')
plt.show()
plt.hist(ordinary['MSRP'])
plt.title('Price Histogram')
plt.show()
plt.hist(ordinary['city mpg'])
plt.title('City mpg Histogram')
plt.show()


It seems like Age,MSRP and city_mpg have a skewed distribution.Lets have a look at our target variable MSRP

In [ ]:
sns.boxplot(x=ordinary['MSRP'])

As we can see above, the price is a positively skewed distribution. We know that one of the assumptions for using LinearRegression is that the variables be normally distributed.
Also,there seem to be some outliers. Lets have a look at them. Usually, we consider an values to be outliers if 
a) Q1 - 1.5IQR 
b) Q3 + 1.5IQR 
where IQR=Q3-Q1 provided the distribution is symmetrical.

In [ ]:
q75, q25 = np.percentile(ordinary['MSRP'], [75 ,25])
iqr = q75 - q25
q75+1.5*q75

Considering that the data is positively skewed and looking at the box plot, we take any values above 140000 to be outliers, for the ordinary segment of cars

In [ ]:
ordinary[ordinary['MSRP']>140000]

When we assigned a grouping based on price, we did so on the basis of Median price per Make.This will work in most of the cases but there may be individual models that can be in a different category. We will remove these outliers before building our models

In [ ]:
ordinary=ordinary[ordinary['MSRP']<140000]

Lets log transform the data to handle the skewness

In [ ]:
ordinary['log_MSRP']=ordinary['MSRP'].apply(lambda x:np.log(1+x))
plt.hist(ordinary['log_MSRP'])
plt.show()
sns.boxplot(x=ordinary['log_MSRP'])

Now, lets have a look at Age. We will check out the relationship of Age with log_MSRP and try different transformations such as log,polynomial etc.

In [ ]:
ordinary['log_Age']=ordinary['Age'].apply(lambda x: np.log(x+1))
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,3))
ax1.set_title('Log Age Histogram')
ax1.hist(ordinary['log_Age'])
ax2.boxplot(ordinary['log_Age'])
ax2.set_title('Log Age Box Plot')
plt.show()

The log_Age distribution appears to be somewhat uniform. Lets have a look at the correlation with age.

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,3))
ax1.scatter(ordinary['Age'],ordinary['log_MSRP'],alpha=.3)
ax1.set_title('Age vs Log MSRP')
ax2.scatter(ordinary['log_Age'],ordinary['log_MSRP'],alpha=.3)
ax2.set_title('Log Age vs Log MSRP')

Lets check with sqrt as well

In [ ]:
ordinary['sqrt_Age']=np.sqrt(ordinary['Age'])
plt.scatter(ordinary['sqrt_Age'],ordinary['log_MSRP'],alpha=.3)
plt.title('sqrt Age vs Log MSRP')
plt.show()

It seems Age and Log MSRP have a linear relationship. Now, lets have a look at city mpg

In [ ]:
sns.lmplot( x="Age", y="log_MSRP", data=ordinary, fit_reg=False, hue='Driven_Wheels', legend=False)
plt.legend(loc='upper right')

In [ ]:
ordinary['log_city mpg']=ordinary['city mpg'].apply(lambda x: np.log(x+1))
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,3))
ax1.hist(ordinary['log_city mpg'])
ax1.set_title('log city mpg histogram')
ax2.boxplot(ordinary['log_city mpg'])
ax2.set_title('log city mpg boxplot')
plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,3))
ax1.scatter(ordinary['city mpg'],ordinary['log_MSRP'],alpha=.1)
ax1.set_title('city mpg vs Log MSRP')
ax2.scatter(ordinary['log_city mpg'],ordinary['log_MSRP'],alpha=.1)
ax2.set_title('Log city mpg vs Log MSRP')

There seems to be some outliers w.r.t city mpg. Lets add legend to the chart and see if we can get some insights.

In [ ]:
sns.lmplot( x="city mpg", y="log_MSRP", data=ordinary, fit_reg=False, hue='Transmission Type', legend=False)
plt.legend(loc='upper right')

The above chart seems interesting as almost all the vehicles which have a DIRECT_DRIVE transmission type seem to have a higher city mpg. 
Lets have a look at the correlations to get a better idea

In [ ]:
corr=ordinary.corr()
corr['log_MSRP'].sort_values(ascending=True)

Age has a stronger correlation with log_MSRP compared to log_Age. For city mpg, we log transform the data as the distribution is close to normal and the correlation is slightly higher. So, the numeric features we are going to use are
Age of Car
Engine HP
log_city mpg 

Lets drop the other numeric_features

In [ ]:
ordinary=ordinary.drop(['log_Age','city mpg'],axis=1)
ordinary.head()

In [ ]:
#Since we have decided to take log MSRP as a target variable, we decide to check relationship between Engine HP, 
#with powers and Log MSRP
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,3))
ax1.scatter(np.power(ordinary['Engine HP'],1/2), ordinary['log_MSRP'],alpha=.3)
ax1.set_title('HP^2 vs Log MSRP')
ax2.scatter(np.power(ordinary['Engine HP'],1/3), ordinary['log_MSRP'],alpha=.3)
ax2.set_title('HP^3 vs Log MSRP')
ax3.scatter(log(1+ordinary['Engine HP']), ordinary['log_MSRP'],alpha=.3)
ax3.set_title('log Engine HP vs Log MSRP')
plt.show()

In [ ]:
corr_p, _ = pearsonr((ordinary['log_MSRP']), ordinary['Engine HP'])
print('Pearson correlation between log Price and Engine HP : %.3f' % corr_p)

corr_p, _ = pearsonr((ordinary['log_MSRP']), np.square(ordinary['Engine HP']))
print('Pearson correlation between log Price and square of Engine HP : %.3f' % corr_p)

corr_p, _ = pearsonr((ordinary['log_MSRP']), np.power(ordinary['Engine HP'],3))
print('Pearson correlation between log Price and cube of Engine HP : %.3f' % corr_p)

corr_p, _ = pearsonr((ordinary['log_MSRP']), np.log(1+ordinary['Engine HP']))
print('Pearson correlation between log Price and log of Engine HP : %.3f' % corr_p)

We find Pearson correlation between log Price and log Engine HP to be the highest and decide to log Engine HP as a numerical feature.


In [ ]:
ordinary['log_Engine HP']=ordinary['Engine HP'].apply(lambda x: np.log(x+1))

Now lets have a look at the categorical features. Since, there are a huge number of models and there maybe very few number of records per model,one-hot encoding the same will result into a huge number of features. We use a few Makes and Models to reduce the number of features.

In [ ]:
fig, (ax1) = plt.subplots(1,1,figsize=(15,5))
Make=ordinary.groupby(['Make'])['MSRP'].mean()
Model=ordinary.groupby(['Model'])['MSRP'].mean()
FuelType=ordinary.groupby(['Engine Fuel Type'])['MSRP'].mean()
Transmission=ordinary.groupby(['Transmission Type'])['MSRP'].mean()
DrivenWheels=ordinary.groupby(['Driven_Wheels'])['MSRP'].mean()
VehicleSize=ordinary.groupby(['Vehicle Size'])['MSRP'].mean()
VehicleStyle=ordinary.groupby(['Vehicle Style'])['MSRP'].mean()
ax1.bar(Make.index,Make.values)
ax1.set_title('Mean MSRP by Make')
plt.sca(ax1)
plt.xticks(rotation=90)

In [ ]:
plt.bar(FuelType.index,FuelType.values)
plt.title('Mean MSRP by Engine Fuel Type')
plt.xticks(rotation=90)
plt.show()
plt.bar(Transmission.index,Transmission.values)
plt.title('Mean MSRP by Transmission Type')
plt.xticks(rotation=90)
plt.show()
plt.bar(DrivenWheels.index,DrivenWheels.values)
plt.title('Mean MSRP by Driven_Wheels')
plt.xticks(rotation=90)
plt.show()
plt.bar(VehicleSize.index,VehicleSize.values)
plt.title('Mean MSRP by Vehicle Size')
plt.xticks(rotation=90)
plt.show()
plt.bar(VehicleStyle.index,VehicleStyle.values)
plt.title('Mean MSRP by Vehicle Style')
plt.xticks(rotation=90)
plt.show()

As we see above the MSRP varies for different values of the features. Hence, we shall keep the above features and one hot encode the same.
Saving the transformed dataframe as a pickle file

In [ ]:
#We take few specific models. This is to restric the number of features.
Makes=['Ford','Chevrolet','Chrysler','Pontiac','Subaru','Hyundai','Honda','Mazda', 'Nissan','Suzuki']
ordinary=ordinary[ordinary.Make.isin(Makes)]

In [ ]:
ordinary.Make.value_counts()

In [ ]:
ordinary_trans='./ordinarydfUSJap.pkl'
with open(ordinary_trans, "wb") as f:
    w = pickle.dump(ordinary,f)

**Data Pre-processing** The numerical variables are scaled with StandardScaler, imputation strategy is used to replace 0 values with mean StratifiedRandomShuffleSplit is done based on Age of car (Curr Year - Year of Car), by creating Age category (Age / 5), and same distribution of Age-category is maintained in Train and Test data. The categorical variables (Engine Cylinders, Engine Fuel type, Transmission type, Vehicle Size, Vehichle Style, Drive wheels, Make) are one-hot encoded and added to the feature vector. The numerical variables considered are : Age of Car (derived from Year of Mfr), Log City mpg, Log Engine HP, based on initial Data Analysis. Two sets of X features are produced, with Make and without Make so the results can be compared. The target variable is Price. Again both Price and log of Price is used and results compared in final predictive Model.

In [ ]:
#Import all necessary libraries
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

Read the pickle files prepared by stratifying the Car Sales Data based on Make and Price. This is necessary since cars belong to different price segments, and including all make and models is not a viable solution, as the range of price is different for the same features for different makes. The stratification details can be found above, which precedes this and outputs the data into .pkl files, based on car segment/price category

In [ ]:
df_ordinary=pd.read_pickle('./ordinarydfUSJap.pkl')
df_ordinary.head()

In [ ]:
df_ordinary["Make"].value_counts()

In [ ]:
print(len(df_ordinary))

We will take the ordinary segment as it has the most data, and fit our model. Once done the same model will be applied to other segments.(For MVP, in actuality the modeling exercise needs to be repeated for each segment, as the relationships may be different)

In [ ]:
#We do some data cleansing, as needed
df_ordinary["Number of Doors"] = df_ordinary["Number of Doors"].replace("?",0)
df_ordinary["Number of Doors"] = df_ordinary["Number of Doors"].astype('float32')
df_ordinary["MSRP"] = df_ordinary["MSRP"].replace("?",0)
df_ordinary["MSRP"] = df_ordinary["MSRP"].astype("float32")
df_ordinary["log_Engine HP"] = df_ordinary["log_Engine HP"].astype("float32")

We check the distribution of Car Sales on the basis of Age of Car, and create Age-cat and check the distribution of the Car data based on Age-cat (Age / 5). We plan to use StratifiedSampling to make sure both Test and Train data represents same distribution of cars based on Age of Car

In [ ]:
df_ordinary["Age"].value_counts()
#create a field Age-cat to divide the data into 5 Age categories, based on the Age of the car
df_ordinary["Age-cat"] = np.ceil(df_ordinary["Age"] / 5)
df_ordinary["Age-cat"].where(df_ordinary["Age-cat"] < 5, 5.0, inplace=True)
#check distribution of Age Cat in the original data
df_ordinary["Age-cat"].value_counts() / len(df_ordinary)

We treat Engine Cylinders, Engine Fuel Type, Transmission Type, Driven_wheels, Vehicle Size, Make and style as Categorical variables based on our Data exploration. We use the LabelBinarizer to fit the variables on the entire set. The actual encoding will be done using the encoded values on the Train and test samples

In [ ]:
car_eng_cyl = df_ordinary["Engine Cylinders"]
encoder_cyl = LabelBinarizer()
encoder_cyl.fit(car_eng_cyl)
print(encoder_cyl.classes_)

car_eng_fuel_type = df_ordinary["Engine Fuel Type"]
encoder_fuel = LabelBinarizer()
encoder_fuel.fit(car_eng_fuel_type)
print(encoder_fuel.classes_)

car_trans_type = df_ordinary["Transmission Type"]
encoder_trans = LabelBinarizer()
encoder_trans.fit(car_trans_type)
print(encoder_trans.classes_)

car_driven_wheels = df_ordinary["Driven_Wheels"]
encoder_wheels = LabelBinarizer()
encoder_wheels.fit(car_driven_wheels)
print(encoder_wheels.classes_)

car_vehicle_size = df_ordinary["Vehicle Size"]
encoder_size = LabelBinarizer()
encoder_size.fit(car_vehicle_size)
print(encoder_size.classes_)

car_make =df_ordinary["Make"]
encoder_make = LabelBinarizer()
encoder_make.fit(car_make)
print(encoder_make.classes_)


car_style =df_ordinary["Vehicle Style"]
encoder_style = LabelBinarizer()
encoder_style.fit(car_style)
print(encoder_style.classes_)


In [ ]:
#We use StratifiedShuffleSplit based on Age-cat, to make sure both train and test data have same distribution of New and Old cars
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

for train_index, test_index in split.split(df_ordinary,df_ordinary["Age-cat"]):
    strat_train_set = df_ordinary.iloc[train_index]
    strat_test_set = df_ordinary.iloc[test_index]

In [ ]:
#check distribution of Age Cat in the train data
strat_train_set["Age-cat"].value_counts() / len(strat_train_set)

In [ ]:
#check distribution of Age Cat in the test data
strat_test_set["Age-cat"].value_counts() / len(strat_test_set)

Create the X and Y variables from the Feature analysis done in Exploration notebook. Repeat the same operations for Train and Test data.

In [ ]:
carSales_X = strat_train_set.copy()
carSales_X = strat_train_set.drop("MSRP", axis=1) # drop labels for training set
carSales_X = strat_train_set.drop("log_MSRP", axis=1) # drop labels for training set
carSales_Y = strat_train_set["log_MSRP"].copy() # use log MSRP as labels for training set, based on data Exploration
carSales_Y_orig = strat_train_set["MSRP"].copy() # use MSRP as labels also for training set, to compare fit based on Log and original Price

carSales_test_X = strat_test_set.copy()
carSales_test_X = strat_test_set.drop("MSRP", axis=1) # drop labels for test set
carSales_test_X = strat_test_set.drop("log_MSRP", axis=1) # drop labels for test set
carSales_test_Y = strat_test_set["log_MSRP"].copy()# use log MSRP as labels for test set, based on data Exploration
carSales_test_Y_orig = strat_test_set["MSRP"].copy()# use MSRP as labels also for test set, to compare fit based on Log and original Price

In [ ]:
carSales_Y = carSales_Y.values.reshape(carSales_Y.shape[0],1)
carSales_test_Y = carSales_test_Y.values.reshape(carSales_test_Y.shape[0],1)
carSales_Y_orig = carSales_Y_orig.values.reshape(carSales_Y_orig.shape[0],1)
carSales_test_Y_orig = carSales_test_Y_orig.values.reshape(carSales_test_Y.shape[0],1)
print(carSales_X.shape)
print(carSales_Y.shape)
print(carSales_Y_orig.shape)
print(carSales_test_X.shape)
print(carSales_test_Y.shape)
print(carSales_test_Y_orig.shape)

Now we need to remove unnecessary columns based on Correlation analysis done in ExplorationNotebook, and do Encoding of Categorical variables. Also we need to do StandardNormalization before applying Regression models.

In [ ]:
carSales_X.head()

We drop all categorical columns after making a copy, and retain only the numerical features of significance

In [ ]:
carSales_X_num = carSales_X
carSales_X_num  = carSales_X_num.drop("Make",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Model",axis=1)
carSales_X_num  = carSales_X_num.drop("Engine Cylinders",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Engine Fuel Type",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Transmission Type",axis=1) # to be treated as categorical var 
carSales_X_num  = carSales_X_num.drop("Driven_Wheels",axis=1) # to be treated as categorical var
carSales_X_num = carSales_X_num.drop("Number of Doors",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Vehicle Style",axis=1) # to be treated as categorical var
carSales_X_num  = carSales_X_num.drop("Engine HP",axis=1)  # since we are taking log of Engine HP,based on Analysis
carSales_X_num = carSales_X_num.drop("Vehicle Size",axis=1) # to be treated as categorical var
carSales_X_num = carSales_X_num.drop("Age-cat",axis=1) # derived column
carSales_X_num = carSales_X_num.drop("sqrt_Age",axis=1) # derived column
carSales_X_num = carSales_X_num.drop("MSRP",axis=1) #Target / label

In [ ]:
carSales_X_num.head()

In [ ]:
#Apply the same transformation on Test data
carSales_test_X_num = carSales_test_X
carSales_test_X_num  = carSales_test_X_num.drop("Make",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Model",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Engine Cylinders",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Engine Fuel Type",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Transmission Type",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Driven_Wheels",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("Number of Doors",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Vehicle Style",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("Vehicle Size",axis=1)
carSales_test_X_num  = carSales_test_X_num.drop("Engine HP",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("Age-cat",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("sqrt_Age",axis=1)
carSales_test_X_num = carSales_test_X_num.drop("MSRP",axis=1)

In [ ]:
carSales_test_X_num.head()

In [ ]:
#We do some data cleansing as needed
carSales_X_num["log_Engine HP"] = carSales_X_num["log_Engine HP"].astype("float32")
carSales_X_num["Age"] = carSales_X_num["Age"].astype("float32")
carSales_X_num.replace('null',np.NaN,inplace=True)
carSales_X_num = pd.DataFrame(carSales_X_num)
carSales_X_num = carSales_X_num.replace('?',0)
carSales_X_num = carSales_X_num.replace('NaN',0)
carSales_X_num = carSales_X_num.replace(np.NaN,0)

carSales_test_X_num["log_Engine HP"] = carSales_test_X_num["log_Engine HP"].astype("float32")
carSales_test_X_num["Age"] = carSales_test_X_num["Age"].astype("float32")
carSales_test_X_num.replace('null',np.NaN,inplace=True)
carSales_test_X_num = pd.DataFrame(carSales_test_X_num)
carSales_test_X_num = carSales_test_X_num.replace('?',0)
carSales_test_X_num = carSales_test_X_num.replace('NaN',0)
carSales_test_X_num = carSales_test_X_num.replace(np.NaN,0)

In [ ]:
m=carSales_X_num.isnull().any()
print(m[m])
m=np.isfinite(carSales_X_num.select_dtypes(include=['float64'])).any()
print(m[m])
m=carSales_test_X_num.isnull().any()
print(m[m])
m=np.isfinite(carSales_test_X_num.select_dtypes(include=['float64'])).any()
print(m[m])

Wherever there are 0 values, we replace by the mean

In [ ]:
imputer = Imputer(missing_values=0,strategy="mean")
imputer.fit(carSales_X_num)
imputer.fit(carSales_test_X_num)

In [ ]:
#Standardize the data using sklearn StandardScaler
scaler = StandardScaler()
train_X = scaler.fit_transform(carSales_X_num)
test_X = scaler.transform(carSales_test_X_num)
print(train_X.shape)

Now add the Categorical variables using one-hot represenation, using the encoder already fit on the entire sample

In [ ]:
car_eng_cyl = carSales_X["Engine Cylinders"]
car_eng_1hot = encoder_cyl.transform(car_eng_cyl)
print(car_eng_1hot.shape)

train_X = np.concatenate((train_X,car_eng_1hot),axis=1)

car_eng_fuel_type = carSales_X["Engine Fuel Type"]
car_fuel_1hot = encoder_fuel.transform(car_eng_fuel_type)
print(car_fuel_1hot.shape)

train_X = np.concatenate((train_X,car_fuel_1hot),axis=1)

car_trans_type = carSales_X["Transmission Type"]
car_trans_1hot = encoder_trans.transform(car_trans_type)
print(car_trans_1hot.shape)

train_X = np.concatenate((train_X,car_trans_1hot),axis=1)

car_driven_wheels = carSales_X["Driven_Wheels"]
car_drive_1hot = encoder_wheels.transform(car_driven_wheels)
print(car_drive_1hot.shape)

train_X = np.concatenate((train_X,car_drive_1hot),axis=1)

car_vehicle_size = carSales_X["Vehicle Size"]
car_size_1hot = encoder_size.transform(car_vehicle_size)
print(car_size_1hot.shape)

train_X = np.concatenate((train_X,car_size_1hot),axis=1)

car_vehicle_style = carSales_X["Vehicle Style"]
car_style_1hot = encoder_style.transform(car_vehicle_style)
print(car_style_1hot.shape)

train_X = np.concatenate((train_X,car_style_1hot),axis=1)

car_make = carSales_X["Make"]
car_make_1hot = encoder_make.transform(car_make)
print(car_make_1hot.shape)

train_X_make = np.concatenate((train_X,car_make_1hot),axis=1)

#We prepare two sets of train X features, with Make and without Make and compare the performance of both
print(train_X.shape)
print(train_X_make.shape)

In [ ]:
car_eng_cyl = carSales_test_X["Engine Cylinders"]
car_eng_1hot = encoder_cyl.transform(car_eng_cyl)
print(car_eng_1hot.shape)

test_X = np.concatenate((test_X,car_eng_1hot),axis=1)

car_eng_fuel_type = carSales_test_X["Engine Fuel Type"]
car_fuel_1hot = encoder_fuel.transform(car_eng_fuel_type)
print(car_fuel_1hot.shape)

test_X = np.concatenate((test_X,car_fuel_1hot),axis=1)

car_trans_type_test = carSales_test_X["Transmission Type"]
car_trans_1hot_test = encoder_trans.transform(car_trans_type_test)
print(car_trans_1hot_test.shape)

test_X = np.concatenate((test_X,car_trans_1hot_test),axis=1)

car_driven_wheels_test = carSales_test_X["Driven_Wheels"]
car_drive_1hot_test = encoder_wheels.transform(car_driven_wheels_test)
print(car_drive_1hot_test.shape)

test_X = np.concatenate((test_X,car_drive_1hot_test),axis=1)

car_vehicle_size_test = carSales_test_X["Vehicle Size"]
car_size_1hot_test = encoder_size.transform(car_vehicle_size_test)
print(car_size_1hot_test.shape)

test_X = np.concatenate((test_X,car_size_1hot_test),axis=1)

car_vehicle_style_test = carSales_test_X["Vehicle Style"]
car_style_1hot_test = encoder_style.transform(car_vehicle_style_test)
print(car_style_1hot_test.shape)

test_X = np.concatenate((test_X,car_style_1hot_test),axis=1)

car_make_test = carSales_test_X["Make"]
car_make_1hot_test = encoder_make.transform(car_make_test)
print(car_make_1hot_test.shape)

test_X_make = np.concatenate((test_X,car_make_1hot_test),axis=1)

print(test_X.shape)
print(test_X_make.shape)

In [ ]:
train_Y = pd.DataFrame(carSales_Y)
m=train_Y.isnull().any()
print(m[m])
m=np.isfinite(train_Y.select_dtypes(include=['float64'])).any()
print(m[m])

train_Y_orig = pd.DataFrame(carSales_Y_orig)
m=train_Y_orig.isnull().any()
print(m[m])
m=np.isfinite(train_Y_orig.select_dtypes(include=['float64'])).any()
print(m[m])

test_Y = pd.DataFrame(carSales_test_Y)
m=test_Y.isnull().any()
print(m[m])
m=np.isfinite(test_Y.select_dtypes(include=['float64'])).any()
print(m[m])

test_Y_orig = pd.DataFrame(carSales_test_Y_orig)
m=test_Y_orig.isnull().any()
print(m[m])
m=np.isfinite(test_Y_orig.select_dtypes(include=['float64'])).any()
print(m[m])

We now take backup of the pre-processed data, so the modeling can be done instantaneously on the pre-processed data at any later point of time

In [ ]:
train_X_ordinary='./train_X_ordUSJap.pkl'
test_X_ordinary='./test_X_ordUSJap.pkl'
train_Y_ordinary='./train_Y_ordUSJap.pkl'
test_Y_ordinary='./test_Y_ordUSJap.pkl'
train_Y_ordinary_orig='./train_Y_ord_origUSJap.pkl'
test_Y_ordinary_orig='./test_Y_ord_origUSJap.pkl'

with open(train_X_ordinary, "wb") as f:
    w = pickle.dump(train_X,f)
with open(test_X_ordinary, "wb") as f:
    w = pickle.dump(test_X,f)
with open(train_Y_ordinary, "wb") as f:
    w = pickle.dump(train_Y,f)
with open(test_Y_ordinary, "wb") as f:
    w = pickle.dump(test_Y,f)
with open(train_Y_ordinary_orig, "wb") as f:
    w = pickle.dump(train_Y_orig,f)
with open(test_Y_ordinary_orig, "wb") as f:
    w = pickle.dump(test_Y_orig,f)
    
train_X_ord_make='./train_X_ord_makeUSJap.pkl'
test_X_ord_make='./test_X_ord_makeUSJap.pkl'

with open(train_X_ord_make, "wb") as f:
    w = pickle.dump(train_X_make,f)
with open(test_X_ord_make, "wb") as f:
    w = pickle.dump(test_X_make,f)

**Predictive Modeling** Various Regression techniques are explored, like LinearRegression, RandomForestRegression, with K-fold cross validation, grid search of parameters, finally Deep Learning techniques, XGBoost is used and the best accuracy is shown. However the LinearRegression model is exposed as WebService, due to certain technical limitations faced in exposing the other models.

In [ ]:
#Import all necessary libraries
import pickle
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from keras.models import Sequential
from keras.layers import Dense   
from keras import optimizers

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

Read the pre-processed pkl files

In [ ]:
train_X=pd.read_pickle('./train_X_ordUSJap.pkl')
test_X=pd.read_pickle('./test_X_ordUSJap.pkl')
train_Y=pd.read_pickle('./train_Y_ordUSJap.pkl') # train Y with log(MSRP)
test_Y=pd.read_pickle('./test_Y_ordUSJap.pkl')
train_Y_orig=pd.read_pickle('./train_Y_ord_origUSJap.pkl') # train Y with MSRP unmodified
test_Y_orig=pd.read_pickle('./test_Y_ord_origUSJap.pkl')

train_X_make=pd.read_pickle('./train_X_ord_makeUSJap.pkl')
test_X_make=pd.read_pickle('./test_X_ord_makeUSJap.pkl')

print(train_X.shape)
print(train_X_make.shape)

**Linear Regression**
We fit Sklearn LinearRegression and use this to make predictions on the test data and check the RMSE Use train X data with make and without make, and Test Y as MSRP as well as Log MSRP and compare the RMSE values on test data

In [ ]:
#fit train data without make info, and log MSRP
lin_reg = LinearRegression()
lin_reg.fit(train_X, train_Y)

#fit train data without make and MSRP, as it
lin_reg_1 = LinearRegression()
lin_reg_1.fit(train_X, train_Y_orig)

#fit train data with make and log MSRP
lin_reg_make = LinearRegression()
lin_reg_make.fit(train_X_make, train_Y)

#fit train data with make and MSRP, as it
lin_reg_make1 = LinearRegression()
lin_reg_make1.fit(train_X_make, train_Y_orig)

In [ ]:
carSales_predictions = lin_reg.predict(test_X)
lin_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
lin_rmse = np.sqrt(lin_mse)
print("rmse without make, log MSRP:"+str(lin_rmse))

carSales_predictions = lin_reg_1.predict(test_X)
lin_mse = mean_squared_error(test_Y, carSales_predictions)
lin_rmse = np.sqrt(lin_mse)
print("rmse without make, MSRP, as is:"+str(lin_rmse))

carSales_predictions = lin_reg_make.predict(test_X_make)
lin_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
lin_rmse = np.sqrt(lin_mse)
print("rmse with make, log MSRP:"+str(lin_rmse))

carSales_predictions = lin_reg_make1.predict(test_X_make)
lin_mse = mean_squared_error(test_Y, carSales_predictions)
lin_rmse = np.sqrt(lin_mse)
print("rmse with make, MSRP, as is:"+str(lin_rmse))

We observe the data with make and log MSRP gives best results, as we found in our analysis. Next we use SGDRegressor from scikit learn and compare the RMSE

In [ ]:
sgd_reg_make = SGDRegressor(max_iter=500,penalty=None,eta0=0.01)
sgd_reg_make.fit(train_X_make, train_Y.values.ravel())

In [ ]:
carSales_predictions_make = sgd_reg_make.predict(test_X_make)
sgd_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions_make))
sgd_rmse = np.sqrt(sgd_mse)
print("SGD RMSE:"+str(sgd_rmse))

**Decision Tree Regressor**

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_X,train_Y)

tree_reg_make = DecisionTreeRegressor()
tree_reg_make.fit(train_X_make,train_Y)

In [ ]:
carSales_predictions = tree_reg.predict(test_X)
tree_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
tree_rmse = np.sqrt(tree_mse)
print("Decision Tree RMSE, without make:"+str(tree_rmse))

In [ ]:
carSales_predictions_make = tree_reg_make.predict(test_X_make)
tree_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions_make))
tree_rmse = np.sqrt(tree_mse)
print("Decision Tree RMSE, with make:"+str(tree_rmse))

In [ ]:
#Lets print few predicted prices and actual prices
print("predicted prices")
print(np.around(np.exp(carSales_predictions_make[0:5])))
print("actual prices")
print(np.exp(test_Y[0:5]))

We find Decision tree with make data included reduces the RMSE and is the best so far. We validate this using K-fold Cross validation with k set to 10

In [ ]:
scores = cross_val_score(tree_reg_make,train_X_make,train_Y,scoring="neg_mean_squared_error",cv=10)
tree_rmse_scores = np.sqrt(-scores)

print("scores:",tree_rmse_scores)
print("mean:",tree_rmse_scores.mean())
print("std dev:",tree_rmse_scores.std())

This shows there is a good fit, and there is very less variation between the folds and the data is dependable. We now try RandomForestRegressor, with default parameters

**RandomForestRegressor**

In [ ]:
forest_reg_make = RandomForestRegressor()
forest_reg_make.fit(train_X_make,train_Y.values.ravel())

In [ ]:
carSales_predictions = forest_reg_make.predict(test_X_make)
forest_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
forest_rmse = np.sqrt(forest_mse)
print("Random Forest Regressor RMSE, with make:"+str(forest_rmse))
print("predicted prices")
print(np.around(np.exp(carSales_predictions[0:5])))
print("actual prices")
print(np.exp(test_Y[0:5]))

This is clearly a better fit compared to DecisionTree, and we validate this with k-fold CV

In [ ]:
forest_scores = cross_val_score(forest_reg_make,train_X_make,train_Y.values.ravel(),scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

print("scores:",forest_rmse_scores)
print("mean:",forest_rmse_scores.mean())
print("std dev:",forest_rmse_scores.std())

We now use **GridSearch **to find the optimum parameters for RandomForestRegression

In [ ]:
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(train_X_make, train_Y.values.ravel())
print("BEST PARAMETERS FOR RANDOM FOREST REGRESSOR IS:")
grid_search.best_params_

In [ ]:
#Fit using best parameters and check
forest_reg_make = RandomForestRegressor(max_features=8,n_estimators=30)
forest_reg_make.fit(train_X_make,train_Y.values.ravel())

In [ ]:
carSales_predictions = forest_reg_make.predict(test_X_make)
forest_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
forest_rmse = np.sqrt(forest_mse)
print("Random Forest Regressor RMSE, with make:"+str(forest_rmse))
print("predicted prices")
print(np.around(np.exp(carSales_predictions_make[0:5])))
print("actual prices")
print(np.exp(test_Y[0:5]))

The observation is : Default parameters gives better results.

**Feature importance** derivation

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
num_attribs = ["Age","City MPG","Engine HP"]
categorical_attribs = [  '0.' , ' 3.',   '4.' ,  '5.' ,  '6.',   '8.' , '10.', '12.'] + ['diesel', 'electric' ,'flex-fuel (unleaded/E85)',
 'flex-fuel (unleaded/natural gas)' ,'natural gas', 'premium unleaded (recommended)', 'premium unleaded (required)',
 'regular unleaded'] + ['AUTOMATED_MANUAL' ,'AUTOMATIC' ,'DIRECT_DRIVE' ,'MANUAL', 'UNKNOWN'] + ['all wheel drive','four wheel drive', 'front wheel drive', 'rear wheel drive'] + ['Compact' ,'Large', 'Midsize']+['2dr Hatchback', '2dr SUV' ,'4dr Hatchback', '4dr SUV', 'Cargo Minivan',
 'Cargo Van', 'Convertible', 'Convertible SUV' ,'Coupe', 'Crew Cab Pickup','Extended Cab Pickup' ,'Passenger Minivan' ,'Passenger Van','Regular Cab Pickup', 'Sedan' ,'Wagon']+['Ford','Chevrolet','Chrysler','Pontiac','Subaru','Hyundai','Honda','Mazda', 'Nissan','Suzuki']
attributes = num_attribs+categorical_attribs
print(sorted(zip(feature_importances, attributes), reverse=True))

This shows the best predictors are Age, Engine HP, Transmission, Fuel Type, City MPG, Drive, Size, MAke, Style. We do retain all parameters in the prediction.

In [ ]:
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(test_X_make)
final_mse = mean_squared_error(np.exp(test_Y), np.exp(final_predictions))
final_rmse = np.sqrt(final_mse)
print("Random Forest Regressor Final RMSE:"+str(final_rmse))
print("predicted prices")
print(np.around(np.exp(final_predictions[0:5])))
print("actual prices")
print(np.exp(test_Y[0:5]))


final_model_scores = cross_val_score(final_model,test_X_make,test_Y.values.ravel(),scoring="neg_mean_squared_error",cv=10)
final_model_scores = np.sqrt(-final_model_scores)

print("scores:",final_model_scores)
print("mean:",final_model_scores.mean())
print("std dev:",final_model_scores.std())

We find the model is fitting quite well. We plot the learning curves for the best model and see the learning curve, by plotting the rmse vs size of training set

In [ ]:
def plot_learning_curves(model, X, y):
    
    train_errors, val_errors = [], []
    for m in range(1, len(X)):
        model.fit(X[:m], y[:m].values.ravel())
        y_train_predict = model.predict(X[:m])
        y_val_predict = model.predict(test_X_make)
        train_errors.append(mean_squared_error(y[:m], y_train_predict))
        val_errors.append(mean_squared_error(test_Y, y_val_predict))

    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.legend(loc="upper right", fontsize=14)   
    plt.xlabel("Training set size", fontsize=14) 
    plt.ylabel("RMSE", fontsize=14)   

plot_learning_curves(final_model, train_X_make, train_Y)
plt.show()


In [ ]:
#We try ElasticNet
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(train_X_make, train_Y)
carSales_predictions = elastic_net.predict(test_X_make)
elastic_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
elastic_rmse = np.sqrt(elastic_mse)
print("Elastic Net RMSE:"+str(elastic_rmse))

In [ ]:
#We try Ridge Regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_reg.fit(train_X_make, train_Y)
carSales_predictions = ridge_reg.predict(test_X_make)
ridge_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
ridge_rmse = np.sqrt(ridge_mse)
print("Ridge RMSE:"+str(ridge_rmse))

In [ ]:
#We try Lasso Regression
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(train_X_make, train_Y)
carSales_predictions = lasso_reg.predict(test_X_make)
lasso_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
lasso_rmse = np.sqrt(lasso_mse)
print("Lasso RMSE:"+str(lasso_rmse))

We find these give worse results. We use GradientBoostingRegressor from sklearn finally

**Gradient Boosting Regressor**

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=8, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(train_X_make, train_Y.values.ravel())
carSales_predictions = gbrt.predict(test_X_make)
gbrt_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
gbrt_rmse = np.sqrt(gbrt_mse)
print("Gradient Boosting Regressor RMSE:"+str(gbrt_rmse))

In [ ]:
gbrt_slow = GradientBoostingRegressor(max_depth=30, n_estimators=200, learning_rate=0.1, random_state=42)
gbrt_slow.fit(train_X_make, train_Y.values.ravel())
carSales_predictions = gbrt_slow.predict(test_X_make)
gbrt_slow_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
gbrt_slow_rmse = np.sqrt(gbrt_slow_mse)
print("Gradient Boosting Regressor SLOW RMSE:"+str(gbrt_slow_rmse))

**GridSearch**

In [ ]:
param_grid = [
    # try 2 (2×2) combinations of hyperparameters
    {'n_estimators': [100,200], 'max_depth': [20, 30]},
    # then try 6 (2×3) combinations with bootstrap set as False
    #{'bootstrap': [False], 'n_estimators': [100,200], 'max_depth': [20, 30, 40]},
  ]

gbrt_reg = GradientBoostingRegressor(random_state=42, learning_rate=0.1)
# train across 5 folds, that's a total of (4)*5=20 rounds of training 
grid_search_gbrt = GridSearchCV(gbrt_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search_gbrt.fit(train_X_make, train_Y.values.ravel())
print("BEST PARAMETERS FOR GRADIENT BOOSTING REGRESSOR IS:")
grid_search_gbrt.best_params_

In [ ]:
gbrt_slow = GradientBoostingRegressor(max_depth=20, n_estimators=100, learning_rate=0.1, random_state=42)
gbrt_slow.fit(train_X_make, train_Y.values.ravel())
carSales_predictions = gbrt_slow.predict(test_X_make)
gbrt_slow_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
gbrt_slow_rmse = np.sqrt(gbrt_slow_mse)
print("Gradient Boosting Regressor BEST RMSE:"+str(gbrt_slow_rmse))

**Deep Learning **technique. We use keras with tensorflow backend 

In [ ]:
model = Sequential()

#We use two hidden layers with 50 and 30 units with Relu activation, and no activation in the output layer, since 
#we want to predict the car price.
model.add(Dense(50,input_dim=(train_X_make.shape[1]),activation='relu'))
model.add(Dense(30,activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
myOptimizer = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=myOptimizer, metrics=['mse'])
history = model.fit(train_X_make, train_Y, epochs=200,  validation_data=(test_X_make,test_Y), batch_size=5, verbose=0)

In [ ]:
plt.plot(history.history['loss'], color = 'blue')
plt.plot(history.history['val_loss'], color=  'red')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
carSales_predictions = model.predict(test_X_make)
dl_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
dl_rmse = np.sqrt(dl_mse)
print("Deep Learning RMSE with two hidden layers:"+str(dl_rmse))

In [ ]:
model = Sequential()
model.add(Dense(50,input_dim=(train_X_make.shape[1]),activation='relu'))
model.add(Dense(20,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1))
model.summary()

In [ ]:
myOptimizer = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0)
model.compile(loss='mean_squared_error', optimizer=myOptimizer, metrics=['mse'])
history = model.fit(train_X_make, train_Y, epochs=300,  validation_data=(test_X_make,test_Y), batch_size=10, verbose=0)

In [ ]:
carSales_predictions = model.predict(test_X_make)
dl_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
dl_rmse = np.sqrt(dl_mse)
print("Deep Learning RMSE with three hidden layers:"+str(dl_rmse))

We now try XGBRegressor, a latest technique. We start by default setting and then use best parameter for n_estimator by trying number of values

In [ ]:
xgb_model = XGBRegressor() 
xgb_model.fit(train_X_make, train_Y)

In [ ]:
carSales_predictions = xgb_model.predict(test_X_make)
xgb_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
xgb_rmse = np.sqrt(xgb_mse)
print("XGB RMSE:"+str(xgb_rmse))

In [ ]:
xgb_model = XGBRegressor(n_estimators=350, max_depth=5) 
xgb_model.fit(train_X_make, train_Y)

In [ ]:
carSales_predictions = xgb_model.predict(test_X_make)
xgb_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
xgb_rmse = np.sqrt(xgb_mse)
print("XGB RMSE, with n_estmators=350,max_depth=5:"+str(xgb_rmse))

In [ ]:
xgb_model = XGBRegressor(n_estimators=350, max_depth=10) 
xgb_model.fit(train_X_make, train_Y)

In [ ]:
carSales_predictions = xgb_model.predict(test_X_make)
xgb_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
xgb_rmse = np.sqrt(xgb_mse)
print("XGB RMSE, with n_estmators=350,max_depth=10:"+str(xgb_rmse))

We conclude the setting of n_estimators=350 and max_depth=5 works best in this case, and this model is the best amongst all the models compared.

We now calculate and display** feature importances** from the XGBoost model

In [ ]:
xgb_model = XGBRegressor(n_estimators=350, max_depth=5) 
xgb_model.fit(train_X_make, train_Y)
print(xgb_model.feature_importances_)
# plot
plt.bar(range(len(xgb_model.feature_importances_)), xgb_model.feature_importances_)
plt.show()

We see that the numerical features, Engine HP, Age, City MPG has the highest importances, followed by categorical features. Among the categorical features we find the Engine Cylinder, and Engine Fuel Type has least importance. Transmission type, and Size, and Make have high importances. We drop the Engine Cylinder from Train and Test data and fit our model again.
We recall the order of Categorical variables is : [ 0. 3. 4. 5. 6. 8. 10. 12.] ['diesel' 'electric' 'flex-fuel (unleaded/E85)' 'flex-fuel (unleaded/natural gas)' 'natural gas' 'premium unleaded (recommended)' 'premium unleaded (required)' 'regular unleaded'] ['AUTOMATED_MANUAL' 'AUTOMATIC' 'DIRECT_DRIVE' 'MANUAL' 'UNKNOWN'] ['all wheel drive' 'four wheel drive' 'front wheel drive' 'rear wheel drive'] ['Compact' 'Large' 'Midsize'] ['2dr Hatchback' '2dr SUV' '4dr Hatchback' '4dr SUV' 'Cargo Minivan' 'Cargo Van' 'Convertible' 'Convertible SUV' 'Coupe' 'Crew Cab Pickup' 'Extended Cab Pickup' 'Passenger Minivan' 'Passenger Van' 'Regular Cab Pickup' 'Sedan' 'Wagon'] ['Ford','Chevrolet','Chrysler','Pontiac','Subaru','Hyundai','Honda','Mazda', 'Nissan','Suzuki']

We drop Engine cylinder from set of features and recalculate RMSE

In [ ]:
print(train_X_make.shape[1])
train_X_make_upd = np.delete(train_X_make,[4,5,6,7,8,9,10,11],1) # we drop the 8 columns after the first 6 numeric ones
print(train_X_make_upd.shape[1])

In [ ]:
test_X_make_upd = np.delete(test_X_make,[4,5,6,7,8,9,10,11],1)

In [ ]:
xgb_model = XGBRegressor(n_estimators=350, max_depth=5) 
xgb_model.fit(train_X_make_upd, train_Y)

In [ ]:
carSales_predictions = xgb_model.predict(test_X_make_upd)
xgb_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
xgb_rmse = np.sqrt(xgb_mse)
print("XGB RMSE, with n_estmators=350,max_depth=5:"+str(xgb_rmse))

We find removing the Engine cylinder categorical variable, does not have any effect on the model. We now try removing the Engine fuel type categorical variable

In [ ]:
print(train_X_make.shape[1])
train_X_make_upd1 = np.delete(train_X_make,[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],1) # we drop the 8 columns after the first 3 numeric ones
print(train_X_make_upd1.shape[1])

In [ ]:
test_X_make_upd1 = np.delete(test_X_make,[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],1)

In [ ]:
xgb_model = XGBRegressor(n_estimators=350, max_depth=5) 
xgb_model.fit(train_X_make_upd1, train_Y)

In [ ]:
carSales_predictions = xgb_model.predict(test_X_make_upd1)
xgb_mse = mean_squared_error(np.exp(test_Y), np.exp(carSales_predictions))
xgb_rmse = np.sqrt(xgb_mse)
print("XGB RMSE, with n_estmators=350,max_depth=5:"+str(xgb_rmse))

We find removing the Engine Fuel Type categorical variable, has no effect on accuracy. 
We decide to drop the Engine Cylinder, and retain Engine Fuel Type based on this analysis from our original set of parameters.

So our final model has the numerical parameters :log Engine HP, Age of Car, log City MPG
and the categorical parameters : Engine Fuel Type,Transmission Type, Driven Wheels, Vehicle Size, Vehicle Style and Make

In [ ]:
# save model to file
pickle.dump(xgb_model, open("./carsales_xgb.pickle.dat", "wb"))